In [1]:
# Multiple Outputs
import numpy as np
import os
import cv2

In [2]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Input, Layer
from keras.layers.advanced_activations import PReLU
#from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [3]:
# Load the TensorBoard notebook extension.
from datetime import datetime
from packaging import version

from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.1.0


## Dataset 

In [4]:
FACES_PATH = '../data/face_detection/faces/'

In [5]:
print(FACES_PATH)

../data/face_detection/faces/


### Read training dataset

In [6]:
training_size = 6000

def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk(FACES_PATH+'pos_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open(FACES_PATH+'coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    #idx=[1,0,3,2]
    idx=[0,1,2,3]
    f_count = 0
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
        f_count = f_count+1
        if f_count == file_count:
            break
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk(FACES_PATH+'neg_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
#images=(np.float32)(images-127.5)/128.0
images=(np.float32)(images)/255

#images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

In [7]:
print('X_train , Image batch shape ', images.shape)
print('y_train , Classification ground true batch shape ' ,np.array(prob).shape)
print('y_train , Coordinates ground true batch shape ', np.array(coordinates).shape)

X_train , Image batch shape  (12000, 12, 12, 3)
y_train , Classification ground true batch shape  (12000, 1, 1, 2)
y_train , Coordinates ground true batch shape  (12000, 1, 1, 4)


## Create X_data for train and validation

In [8]:
X_data = images

In [9]:
print('X_data shape',X_data.shape)

X_data shape (12000, 12, 12, 3)


In [10]:
del images

## Create "y_data" for train and validation

In [11]:
y_data = np.concatenate((np.array(prob), np.array(coordinates)), axis=3)

In [12]:
print('y_data shape',y_data.shape)

y_data shape (12000, 1, 1, 6)


In [13]:
print('y_data Classification shape', y_data[:,:,:,:2].shape)
print('y_data Coordinate shape',y_data[:,:,:,2:].shape)

y_data Classification shape (12000, 1, 1, 2)
y_data Coordinate shape (12000, 1, 1, 4)


## Divide dataset to "train', "val" and "test"

In [14]:
def load_data(X, y, training_prec = 0.7, val_prec = 0.1, test_prec = 0.2):
        data_length = len(X)
        num_training = np.int(data_length * training_prec)
        num_validation = np.int(data_length * val_prec)
        
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]
        mask = range(num_training, num_training + num_validation)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training + num_validation, data_length)
        X_test = X[mask]
        y_test = y[mask]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


In [15]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(X_data, y_data)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (8400, 12, 12, 3)
Train labels shape:  (8400, 1, 1, 6) float64
Validation data shape:  (1200, 12, 12, 3)
Validation labels shape:  (1200, 1, 1, 6)
Test data shape:  (2400, 12, 12, 3)
Test labels shape:  (2400, 1, 1, 6)


In [16]:
log_base_dir = "./logs/p-net-benchmark"
os.makedirs(log_base_dir, exist_ok=True)

In [17]:
print(log_base_dir)

./logs/p-net-benchmark


In [18]:
%load_ext tensorboard
%tensorboard --logdir {log_base_dir} --port=8008

Reusing TensorBoard on port 8008 (pid 21680), started 0:14:21 ago. (Use '!kill 21680' to kill it.)

# Build P-Net Keras model

## 1. ReLU Activation function

In [19]:
def Relu_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='relu')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='relu')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='relu')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net relu.png')
    
    return model

In [20]:
ReLU_model = Relu_PNet()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 10, 10)   280         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 5, 5, 10)     0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3, 3, 16)     1456        max_pooling2d[0][0]              
______________________________________________________________________________________________

In [21]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=('logs\\p-net-benchmark\\ReLU'+datetime.now().strftime("%Y%m%d-%H%M%S")))

In [22]:
ReLU_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 4s 468us/sample - loss: 0.5725 - classification_loss: 0.5496 - bbox_loss: 0.0462 - classification_accuracy: 0.7368 - bbox_mse: 0.0921 - val_loss: 0.5165 - val_classification_loss: 0.4976 - val_bbox_loss: 0.0411 - val_classification_accuracy: 0.7450 - val_bbox_mse: 0.0809
Epoch 2/60
8400/8400 [==============================] - 1s 173us/sample - loss: 0.3882 - classification_loss: 0.3735 - bbox_loss: 0.0292 - classification_accuracy: 0.8443 - bbox_mse: 0.0583 - val_loss: 0.3461 - val_classification_loss: 0.3328 - val_bbox_loss: 0.0272 - val_classification_accuracy: 0.8633 - val_bbox_mse: 0.0541
Epoch 3/60
8400/8400 [==============================] - 2s 192us/sample - loss: 0.3345 - classification_loss: 0.3218 - bbox_loss: 0.0258 - classification_accuracy: 0.8694 - bbox_mse: 0.0515 - val_loss: 0.3191 - val_classification_loss: 0.3072 - val_bbox_loss: 0.0259 - val_classification_accuracy

Epoch 26/60
8400/8400 [==============================] - 1s 178us/sample - loss: 0.1143 - classification_loss: 0.1095 - bbox_loss: 0.0122 - classification_accuracy: 0.9574 - bbox_mse: 0.0242 - val_loss: 0.1654 - val_classification_loss: 0.1592 - val_bbox_loss: 0.0139 - val_classification_accuracy: 0.9417 - val_bbox_mse: 0.0277
Epoch 27/60
8400/8400 [==============================] - 2s 200us/sample - loss: 0.1032 - classification_loss: 0.0973 - bbox_loss: 0.0115 - classification_accuracy: 0.9660 - bbox_mse: 0.0230 - val_loss: 0.1717 - val_classification_loss: 0.1662 - val_bbox_loss: 0.0138 - val_classification_accuracy: 0.9450 - val_bbox_mse: 0.0273
Epoch 28/60
8400/8400 [==============================] - 2s 222us/sample - loss: 0.1012 - classification_loss: 0.0971 - bbox_loss: 0.0115 - classification_accuracy: 0.9646 - bbox_mse: 0.0229 - val_loss: 0.1874 - val_classification_loss: 0.1805 - val_bbox_loss: 0.0144 - val_classification_accuracy: 0.9408 - val_bbox_mse: 0.0286
Epoch 29/60
8

Epoch 51/60
8400/8400 [==============================] - 2s 293us/sample - loss: 0.0504 - classification_loss: 0.0457 - bbox_loss: 0.0093 - classification_accuracy: 0.9842 - bbox_mse: 0.0187 - val_loss: 0.1655 - val_classification_loss: 0.1589 - val_bbox_loss: 0.0127 - val_classification_accuracy: 0.9525 - val_bbox_mse: 0.0249
Epoch 52/60
8400/8400 [==============================] - 2s 285us/sample - loss: 0.0509 - classification_loss: 0.0461 - bbox_loss: 0.0093 - classification_accuracy: 0.9836 - bbox_mse: 0.0186 - val_loss: 0.1701 - val_classification_loss: 0.1621 - val_bbox_loss: 0.0119 - val_classification_accuracy: 0.9525 - val_bbox_mse: 0.0239
Epoch 53/60
8400/8400 [==============================] - 2s 264us/sample - loss: 0.0393 - classification_loss: 0.0347 - bbox_loss: 0.0087 - classification_accuracy: 0.9893 - bbox_mse: 0.0174 - val_loss: 0.1590 - val_classification_loss: 0.1534 - val_bbox_loss: 0.0120 - val_classification_accuracy: 0.9558 - val_bbox_mse: 0.0238
Epoch 54/60
8

## 2. TanH Activation function

In [23]:
def TanH_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='tanh')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='tanh')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='tanh')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net tanh.png')
    
    return model

In [24]:
TanH_model = TanH_PNet()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 10, 10, 10)   280         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 10)     0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 3, 3, 16)     1456        max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [25]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=('logs\\p-net-benchmark\\TanH'+datetime.now().strftime("%Y%m%d-%H%M%S")))

In [26]:
TanH_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 4s 534us/sample - loss: 0.5664 - classification_loss: 0.5424 - bbox_loss: 0.0476 - classification_accuracy: 0.7424 - bbox_mse: 0.0956 - val_loss: 0.4873 - val_classification_loss: 0.4687 - val_bbox_loss: 0.0360 - val_classification_accuracy: 0.7758 - val_bbox_mse: 0.0714
Epoch 2/60
8400/8400 [==============================] - 2s 280us/sample - loss: 0.4534 - classification_loss: 0.4362 - bbox_loss: 0.0317 - classification_accuracy: 0.8125 - bbox_mse: 0.0633 - val_loss: 0.4443 - val_classification_loss: 0.4294 - val_bbox_loss: 0.0299 - val_classification_accuracy: 0.8250 - val_bbox_mse: 0.0600
Epoch 3/60
8400/8400 [==============================] - 1s 168us/sample - loss: 0.4029 - classification_loss: 0.3885 - bbox_loss: 0.0273 - classification_accuracy: 0.8339 - bbox_mse: 0.0545 - val_loss: 0.3779 - val_classification_loss: 0.3641 - val_bbox_loss: 0.0265 - val_classification_accuracy

8400/8400 [==============================] - 2s 262us/sample - loss: 0.2036 - classification_loss: 0.1954 - bbox_loss: 0.0171 - classification_accuracy: 0.9227 - bbox_mse: 0.0341 - val_loss: 0.2301 - val_classification_loss: 0.2204 - val_bbox_loss: 0.0185 - val_classification_accuracy: 0.8992 - val_bbox_mse: 0.0367classification_loss: 0.1927 - bbox_loss: 0.0170 - classification_accuracy: 0.9239 - bbox
Epoch 26/60
8400/8400 [==============================] - 2s 253us/sample - loss: 0.2037 - classification_loss: 0.1953 - bbox_loss: 0.0172 - classification_accuracy: 0.9217 - bbox_mse: 0.0342 - val_loss: 0.2408 - val_classification_loss: 0.2323 - val_bbox_loss: 0.0187 - val_classification_accuracy: 0.9050 - val_bbox_mse: 0.0375
Epoch 27/60
8400/8400 [==============================] - 2s 205us/sample - loss: 0.2033 - classification_loss: 0.1950 - bbox_loss: 0.0173 - classification_accuracy: 0.9211 - bbox_mse: 0.0345 - val_loss: 0.2311 - val_classification_loss: 0.2216 - val_bbox_loss: 0.018

Epoch 50/60
8400/8400 [==============================] - 1s 161us/sample - loss: 0.0981 - classification_loss: 0.0915 - bbox_loss: 0.0126 - classification_accuracy: 0.9685 - bbox_mse: 0.0252 - val_loss: 0.1595 - val_classification_loss: 0.1527 - val_bbox_loss: 0.0152 - val_classification_accuracy: 0.9425 - val_bbox_mse: 0.0304
Epoch 51/60
8400/8400 [==============================] - 1s 165us/sample - loss: 0.0936 - classification_loss: 0.0871 - bbox_loss: 0.0123 - classification_accuracy: 0.9700 - bbox_mse: 0.0247 - val_loss: 0.1505 - val_classification_loss: 0.1426 - val_bbox_loss: 0.0147 - val_classification_accuracy: 0.9458 - val_bbox_mse: 0.0293
Epoch 52/60
8400/8400 [==============================] - 1s 173us/sample - loss: 0.0882 - classification_loss: 0.0819 - bbox_loss: 0.0120 - classification_accuracy: 0.9736 - bbox_mse: 0.0240 - val_loss: 0.1500 - val_classification_loss: 0.1425 - val_bbox_loss: 0.0146 - val_classification_accuracy: 0.9483 - val_bbox_mse: 0.0291
Epoch 53/60
8

## 2. Sigmoid Activation function

In [27]:
def sigmoid_PNet():
    
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

    # input layer
    visible = Input(shape=(12,12,3))
    
    # CNN Stage 1
    conv1 = Conv2D(10, kernel_size=(3,3), activation='sigmoid')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    #CNN Stage 2
    conv2 = Conv2D(16, kernel_size=(3,3), activation='sigmoid')(pool1)
    
    # CNN stage 3
    conv3 = Conv2D(32, kernel_size=(3,3), activation='sigmoid')(conv2)
    
    # output 
    pred_classification = Conv2D(2, kernel_size=(1,1), activation='softmax', name='classification')(conv3)
    pred_bbox = Conv2D(4, kernel_size=(1,1), name='bbox')(conv3)
    
    model = Model(inputs=visible, outputs=[pred_classification, pred_bbox])
                  
 
    #compute the loss function over bounding box 
    bbox_loss = tf.keras.losses.MeanSquaredError()
    
    # Define bbox loss : MSE(bounding_bbox) * y_classification[1] (...ignore if no face)
    # Actually, 
    # we could use 'mse' but because bbox error is "zero" if "no face", we need to multiply 'mse' by "y_classification"  
    def bbox_loss_fn():
            #Create boox loss function 
        def loss(y_true,y_pred):
            return (bbox_loss(pred_bbox, y_bbox) * y_classification[:,:,:,1])
        # Return a function
        return loss
    
 
    # create placeholder for targets
    y_classification = tf.keras.backend.placeholder(dtype='float32', shape=pred_classification.shape) # shapes of output1 your target has
    y_bbox = tf.keras.backend.placeholder(dtype='float32', shape=pred_bbox.shape) # shapes of output2 your target has
    
    # Set optimizer
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    
    model.compile(optimizer=adam, 
                  loss ={'classification': 'binary_crossentropy',
                         'bbox': bbox_loss_fn()},
                  loss_weights = {'classification': 1.0, 
                                  'bbox': 0.5},
                  target_tensors=[y_classification,y_bbox],
                  metrics={'classification': 'accuracy',
                           'bbox': 'mse'})
    # summarize layers
    print(model.summary())
    
    # plot graph
    plot_model(model, to_file='MTCNN P-Net sigmoid.png')
    
    return model

In [28]:
sigmoid_model = sigmoid_PNet()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12, 12, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 10, 10, 10)   280         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 5, 5, 10)     0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 3, 3, 16)     1456        max_pooling2d_2[0][0]            
____________________________________________________________________________________________

In [29]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=('logs\\p-net-benchmark\\Sigmoid'+datetime.now().strftime("%Y%m%d-%H%M%S")))

In [30]:
sigmoid_model.fit(X_train, [y_train[:,:,:,:2],y_train[:,:,:,2:]], batch_size=64, epochs=60,
            validation_data=(X_val, [y_val[:,:,:,:2],y_val[:,:,:,2:]]),
            callbacks=[tensorboard_callback])

Train on 8400 samples, validate on 1200 samples
Epoch 1/60
8400/8400 [==============================] - 4s 488us/sample - loss: 0.7316 - classification_loss: 0.6963 - bbox_loss: 0.0702 - classification_accuracy: 0.5017 - bbox_mse: 0.1422 - val_loss: 0.7183 - val_classification_loss: 0.6926 - val_bbox_loss: 0.0514 - val_classification_accuracy: 0.4975 - val_bbox_mse: 0.1021
Epoch 2/60
8400/8400 [==============================] - 1s 171us/sample - loss: 0.7178 - classification_loss: 0.6925 - bbox_loss: 0.0507 - classification_accuracy: 0.5077 - bbox_mse: 0.1013 - val_loss: 0.7139 - val_classification_loss: 0.6885 - val_bbox_loss: 0.0511 - val_classification_accuracy: 0.5008 - val_bbox_mse: 0.1010
Epoch 3/60
8400/8400 [==============================] - 1s 171us/sample - loss: 0.7030 - classification_loss: 0.6785 - bbox_loss: 0.0494 - classification_accuracy: 0.5865 - bbox_mse: 0.0981 - val_loss: 0.6796 - val_classification_loss: 0.6552 - val_bbox_loss: 0.0481 - val_classification_accuracy

Epoch 26/60
8400/8400 [==============================] - 1s 155us/sample - loss: 0.3602 - classification_loss: 0.3481 - bbox_loss: 0.0240 - classification_accuracy: 0.8579 - bbox_mse: 0.0477 - val_loss: 0.3471 - val_classification_loss: 0.3358 - val_bbox_loss: 0.0233 - val_classification_accuracy: 0.8625 - val_bbox_mse: 0.0465
Epoch 27/60
8400/8400 [==============================] - 2s 206us/sample - loss: 0.3552 - classification_loss: 0.3425 - bbox_loss: 0.0235 - classification_accuracy: 0.8586 - bbox_mse: 0.0469 - val_loss: 0.3503 - val_classification_loss: 0.3389 - val_bbox_loss: 0.0239 - val_classification_accuracy: 0.8617 - val_bbox_mse: 0.0470
Epoch 28/60
8400/8400 [==============================] - 1s 178us/sample - loss: 0.3568 - classification_loss: 0.3446 - bbox_loss: 0.0236 - classification_accuracy: 0.8583 - bbox_mse: 0.0471 - val_loss: 0.3425 - val_classification_loss: 0.3305 - val_bbox_loss: 0.0234 - val_classification_accuracy: 0.8642 - val_bbox_mse: 0.0464
Epoch 29/60
8

8400/8400 [==============================] - 3s 299us/sample - loss: 0.3137 - classification_loss: 0.3049 - bbox_loss: 0.0210 - classification_accuracy: 0.8768 - bbox_mse: 0.0418 - val_loss: 0.2997 - val_classification_loss: 0.2889 - val_bbox_loss: 0.0202 - val_classification_accuracy: 0.8783 - val_bbox_mse: 0.0405
Epoch 51/60
8400/8400 [==============================] - 2s 294us/sample - loss: 0.3095 - classification_loss: 0.2999 - bbox_loss: 0.0206 - classification_accuracy: 0.8798 - bbox_mse: 0.0413 - val_loss: 0.3028 - val_classification_loss: 0.2927 - val_bbox_loss: 0.0209 - val_classification_accuracy: 0.8817 - val_bbox_mse: 0.0412
Epoch 52/60
8400/8400 [==============================] - 2s 285us/sample - loss: 0.3133 - classification_loss: 0.3022 - bbox_loss: 0.0209 - classification_accuracy: 0.8769 - bbox_mse: 0.0419 - val_loss: 0.2893 - val_classification_loss: 0.2786 - val_bbox_loss: 0.0196 - val_classification_accuracy: 0.8883 - val_bbox_mse: 0.0394
Epoch 53/60
8400/8400 [==

### Test the PNet  to ensure that the implementation does not crash and produces outputs of the expected shape.
Pnet will output are:
1. Face classification,  size (batch,1,1,2) for 2 calss classification, "Face", and "Not face"
2. Bounding box  (batch,1,1,4) for 4 boundind box corrdinates (x,y,w,h)

In [ ]:
model.save('P-Net-relu.h5')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics.head()

In [ ]:
metrics[['loss', 'val_loss']].plot(figsize=(8,5))
plt.title('P-Net model loss vs. epochs')

In [ ]:
metrics[['classification_accuracy', 'val_classification_accuracy']].plot(figsize=(8,5))
plt.title('P-Net model accuracy vs. epochs')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
predictions = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

In [ ]:
score = predictions[0]
bbox = predictions[1]

In [ ]:
score = np.squeeze(score)
bbox = np.squeeze(bbox)

In [ ]:
y_test_score = np.squeeze(y_test[:,:,:,:2])
y_test_bbox = np.squeeze(y_test[:,:,:,2:])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_test_score, np.round(score)))

In [ ]:
print(classification_report(y_test_score, np.round(score)))

In [ ]:
print(confusion_matrix(y_test_score[:,1:2], np.round(score[:,1:2])))

In [ ]:
print(score[0:10,1:])

In [ ]:
def plot_testset(index):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(2, 2))
    ax.imshow(X_test[index])
    plt.title(score[index,1:])
    # Create a Rectangle patch
    x = round(12*bbox[index,0])
    y = round(12*bbox[index,1])
    w = round(12*bbox[index,2]) - x
    h = round(12*bbox[index,3]) - y
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)


In [ ]:
plot_testset(0)

In [ ]:
with open('P-Net-Relu-test_data_input.npy', 'wb') as f:
    np.save(f, np.transpose(X_test[0], (2,0,1)))


In [ ]:
# Save prediction for future use (NN lite)
with open('P-Net-Relu-test-data_predictions-classifications.npy', 'wb') as f:
    np.save(f, score[0,:])

with open('P-Net-Relu-test-data_predictions-bbox.npy', 'wb') as f:
    np.save(f, bbox[0,:])

    
# Save labels for future use (NN lite)
with open('P-Net-Relu-test-data_y_Labels_classifications.npy', 'wb') as f:
    np.save(f, y_test_score[0,:])

    
    # Save labels for future use (NN lite)
with open('P-Net-Relu-test-data_y_Labels_bbox.npy', 'wb') as f:
    np.save(f, y_test_bbox[0,:])


In [ ]:
abc = np.load('P-Net-Relu-test_data_input.npy')

In [ ]:
abc.shape